# Experimentación
En el presente cuaderno de Jupiter se corren los experimentos planteados y se escriben los resultados en archivos .CSV en la carpeta de resultados.

In [108]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

Ahora, se leen los datasets con las instancias creadas:

In [130]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

df_densidad_alta = pd.read_csv("instancias/densidad-alta/indice.csv");
df_densidad_baja = pd.read_csv("instancias/densidad-baja/indice.csv");
df_mejor_caso_bt = pd.read_csv("instancias/mejor-caso-BT/indice.csv");
df_mal_caso_bt = pd.read_csv("instancias/mal-caso-BT/indice.csv");
df_peor_caso_bt_o = pd.read_csv("instancias/peor-caso-BT-O/indice.csv");
df_peor_caso_bt_f = pd.read_csv("instancias/peor-caso-BT-F/indice.csv");
df_dinamica = pd.read_csv("instancias/dinamica/indice.csv");
df_dp_vs_bt = pd.read_csv("instancias/dinamica_VS_bt/indice.csv");

La siguiente función sirve para correr el código de cada algotimo sobe una instancia.
- FB: Fuerza bruta
- BT: Backtracking con ambas podas.
- BT-F: Backtracking únicamente usando poda por factibilidad.
- BT-O: Backtracking únicamente usando por optimalidad.
- DP: Programación dinámica.

In [31]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../jambo-tubos", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code is not 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion;

In [6]:
def experimentar(experimentos, nombre):
    columnas = ["dataset", "n", "R", "metodo", "tiempo"];
    filas = [];
    numero = 1
    T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
    for experimento in experimentos:
        # Voy mostrando que experimento se esta ejecutando.
        clear_output(wait=True)
        display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
        numero += 1

        # Ejecutamos el experimento T veces y obtenemos la mediana.
        tiempos = []
        for i in range(0, T):
            print(experimento[3], experimento[4])
            tiempos.append(correr_experimento(experimento[3], experimento[4]));
        tiempo = np.median(tiempos);
        filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo]);
    df_resultado = pd.DataFrame(filas, columns=columnas);
    df_resultado.to_csv(F"resultados/resultado_{nombre}.csv", index=False, header=True);

## Experimentos
A continuación se crea una tabla con las ejecuciones y sus respectivos tiempos.

## Experimento 1:
    Complejidad teorica de FB y comparacion con BT.

In [ ]:
experimentos = [];
for n in range(1, 31):
    fila_n = df_densidad_alta.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["R"], "FB", fila_n["archivo"]]);

for n in range(1, 31):
    fila_n = df_densidad_baja.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["R"], "FB", fila_n["archivo"]]);
    
for n in range(1, 31):
    fila_n = df_densidad_alta.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["R"], "BT", fila_n["archivo"]]);

for n in range(1, 31):
    fila_n = df_densidad_baja.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["R"], "BT", fila_n["archivo"]]);
experimentar(experimentos,"1-FBvsBT")
experimentos = [];  

## Experimento 2: 
    Mejor Caso vs Mal Caso de Backtracking

In [41]:
experimentos = []; 
for n in range(0, df_mejor_caso_bt.shape[0]):
    fila_n = df_mejor_caso_bt.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["R"], "BT", fila_n["archivo"]]);
for n in range(0, df_mal_caso_bt.shape[0]):
    fila_n = df_mal_caso_bt.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["R"], "BT", fila_n["archivo"]]);
experimentar(experimentos,"2-BT-MejorCasoVSMalCaso")
experimentos = []; 

NameError: name 'df_densidad_baja_bt' is not defined

## Experimento 3
    Peor caso ambas podas backtraking y comparacion baja densiada vs alta densidad.

In [65]:
experimentos = []; 
for n in range(0, df_peor_caso_bt_o.shape[0]):
    fila_n = df_peor_caso_bt_o.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["R"], "BT-O", fila_n["archivo"]]);
for n in range(0, df_peor_caso_bt_f.shape[0]):
    fila_n = df_peor_caso_bt_f.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["R"], "BT-F", fila_n["archivo"]]);
for n in range(0, df_densidad_baja.shape[0]):
    fila_n = df_densidad_baja.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["R"], "BT", fila_n["archivo"]]);
for n in range(0, df_densidad_alta.shape[0]):
    fila_n = df_densidad_alta.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["R"], "BT", fila_n["archivo"]]);

experimentar(experimentos,"3-BT-PeorCasoPodas_Densidades")
experimentos = []; 

'Experimento: 260/260'

## Experimento 4
    DP para n fijo y R variable.

In [132]:
#Correr el dataset para las instancias de dinamica con DP
experimentos = []; 
for i in range(0, df_dinamica.shape[0]):
    fila = df_dinamica.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "DP", fila["archivo"]]);
experimentar(experimentos,"4-Dinamica")
experimentos = []; 

'Experimento: 224/224'

In [131]:
#Correr el dataset para las instancias de dinamica con BT
experimentos = []; 
for i in range(0, df_dp_vs_bt[(df_dp_vs_bt["dataset"]=="BT")].shape[0]):
    fila = df_dp_vs_bt[(df_dp_vs_bt["dataset"]=="BT")].iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "BT", fila["archivo"]]);
for i in range(0, df_dp_vs_bt[(df_dp_vs_bt["dataset"]=="DP")].shape[0]):
    fila = df_dp_vs_bt[(df_dp_vs_bt["dataset"]=="DP")].iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "DP", fila["archivo"]]);
experimentar(experimentos,"4-Dinamica-BT-DP")
experimentos = []; 

'Experimento: 112/112'